## Notebook for reading in data and cleansing data, and dumping data with pickle

### Imports and settings

In [1]:
import pandas as pd

import sklearn
import pickle

GOOD_REVIEW_LIMIT = 3.5

### Reading in data

In [2]:
random_reviews = pd.read_csv(r'./Yelp_data/1000_random_reviews.csv', parse_dates=["date"])
bow1000 = pd.read_csv(r'./Yelp_data/bow1000.csv') 

In [3]:
random_reviews.head()

,business_id,cool,date,funny,review_id,stars,text,useful,user_id
0,KW5edXlPGeZ0cx7MiJyamw,0,2017-03-07 02:03:24,0,nkvJghZnOkZy4HAo87ouSA,5,"A stellar cafe!!! Very classy, warm & inviting...",0,PH2Ly-LUDc5i-eW_97MlNw
1,oxwGyA17NL6c5t1Etg5WgQ,0,2017-04-13 04:03:48,0,7KrKzKgUdflicfUs_bWNUQ,1,"Can you say ""Bait and Switch?"" A couple of da...",1,EnSUGj7a7FocW3blBj_hEw
2,bKxKPGmkwH-Q-GSbt7chNA,1,2013-08-11 20:05:49,1,f54iEgnNZEE-JPcdVuNXSQ,2,"When first walking in to Stingray Sushi, a low...",3,aY4FqMsNXBirS72lM_CdwQ
3,q5viyMO6EtJMmmarb4YmIA,0,2015-01-09 23:57:08,0,AtKH9dzGHIKoyAhOLcVUIw,4,A very beautiful restaurant in the Hardrock ca...,2,CTaZ3ebFGQMHaLZhnO2EuA
4,BS7PU7REvJZ7hVSHA6z8wQ,0,2017-09-03 22:40:45,0,C5vaxaN3IZvmBpvcv0uYXw,2,I've been here twice. The first time was a pre...,1,Kya8CWymuq437rxhHyiM2g


In [7]:
bow1000.head()

,able,absolutely,accommodating,across,actually,add,added,additional,advice,afternoon,...,writing,wrong,year,years,yelp,yes,yet,young,yum,yummy
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
1,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
2,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
3,0,0,0,0,1,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,0
4,0,0,0,0,0,0,0,0,0,0,...,0,0,0,1,0,0,0,0,0,0


## Adding good-bad reviews column
Categorize the data in 1000 random reviews.csv into two different classes: 

In [5]:
# poistetaan ensin vanha, jos on jo olemassa
if CATEGORY_COLUMN_NAME in random_reviews.columns:
  random_reviews.drop([CATEGORY_COLUMN_NAME], axis=1)
def label_category (row):
   if row['stars'] >= GOOD_REVIEW_LIMIT : #3.5
      return 1 #'good'
   return 0 #'poor'
random_reviews[CATEGORY_COLUMN_NAME] = random_reviews.apply (lambda row: label_category(row), axis=1)

## Dumping files with pickle

In [6]:
with open("random_reviews.p", "wb") as f:
    pickle.dump( random_reviews, f )

with open("bow1000.p", "wb") as f:
    pickle.dump( bow1000, f )